In [7]:
import json
import pandas as pd
import geopandas as gpd
import numpy as np
import database as db
from munging import get_distinct, get_county, get_centroid, get_rotation
from apsim.daymet import create_excel_met, create_all_excel_met
from apsim.apsim_input_writer import get_date, add_management_year, create_mukey_county_runs
from apsim.apsim_output_parser import parse_all_output, parse_summary_output
from budgets.budget_calcs import Budget
import traceback
import folium
import shapely
import seaborn as sns

In [2]:
cfs_mgmt = json.loads( open( 'crop_jsons/cfs.json', 'r' ).read() )
cc_mgmt = json.loads( open( 'crop_jsons/cc.json', 'r' ).read() )
sfc_mgmt = json.loads( open( 'crop_jsons/sfc.json', 'r' ).read() )

In [3]:
dbconn = db.connect_to_db('database.ini')
raccoon_2018 = 'raccoon.raccoon_clu_ssurgo_2018'
#raccoon = pd.read_sql(f"SELECT * FROM {raccoon_2018} LIMIT 200;", dbconn)
#greene = pd.read_sql(f"SELECT * FROM {raccoon_2018} WHERE County='Greene';", dbconn)
#greene_soils = pd.read_sql(f"SELECT DISTINCT mukey FROM {raccoon_2018} where County='Greene';", dbconn)
#field_25 = pd.read_sql(f"SELECT * FROM {raccoon_2018} WHERE clukey=2515723;", dbconn)

### Get the county centroid for creating APSIM met files

In [ ]:
#buena_vista = get_county(dbconn, 'raccoon.raccoon_clu_ssurgo_2018', 'IA021', "wkb_geometry")
#bv_centroid = get_centroid(buena_vista, 'fips', "wkb_geometry")
# greene = get_county(dbconn, 'raccoon.raccoon_clu_ssurgo_2018', 'IA073', "wkb_geometry")
# greene_centroid = get_centroid(greene, 'fips', "wkb_geometry")
# create_excel_met(greene_centroid[0], greene_centroid[1], 2012, 2019, 'greene')

### Step 3: Get all distinct counties and create met files for all county geometry lat/lon

In [ ]:
fips = get_distinct(dbconn, raccoon_2018, 'county')
# create_all_met(dbconn, fips, raccoon_2018)
# create_all_excel_met(dbconn, fips, raccoon_2018)

In [ ]:
fips

### Get the soil properties for each individual mukey

In [ ]:
#field_25 = pd.read_sql(f"SELECT * FROM {raccoon_2018} WHERE clukey=2515723;", dbconn)
#greene_clu = pd.read_sql(f"SELECT * FROM {raccoon_2018} WHERE County='Greene';", dbconn)

### Create APSIM files for the unique soils for a given county

In [4]:
greene_soils = pd.read_sql(f"SELECT DISTINCT mukey FROM {raccoon_2018} where County='Greene' LIMIT 5;", dbconn)
#greene_soils = pd.read_sql(f"SELECT DISTINCT mukey FROM {raccoon_2018} where County='Greene';", dbconn)
greene_soils_list = list(greene_soils['mukey'])

In [5]:
greene_soils_list

['703831', '403511', '406211', '403455', '406372']

In [8]:
create_mukey_county_runs(greene_soils_list, dbconn, 'sfc', 'Greene', 'IA073', sfc_mgmt=sfc_mgmt, cfs_mgt=cfs_mgmt, cc_mgmt=cc_mgmt)
create_mukey_county_runs(greene_soils_list, dbconn, 'cc', 'Greene', 'IA073', sfc_mgmt=sfc_mgmt, cfs_mgt=cfs_mgmt, cc_mgmt=cc_mgmt)
create_mukey_county_runs(greene_soils_list, dbconn, 'cfs', 'Greene', 'IA073', sfc_mgmt=sfc_mgmt, cfs_mgt=cfs_mgmt, cc_mgmt=cc_mgmt)

Soil 703831 not found
Soil 703831 not found
Soil 703831 not found


In [ ]:
soil_df = pd.DataFrame()
for i in greene_soils_list:
    soil_query = '''select * from api.get_soil_properties( array[{}]::text[] )'''.format( i )
    soil_i_df = pd.read_sql( soil_query, dbconn )
    if soil_i_df.empty:
        continue
    else:
        soil_df = soil_df.append(soil_i_df)

In [ ]:
soil_df

In [ ]:
db_schema = 'raccoon'
db_table = 'greene_soils' 
soil_df.to_sql(
con = dbconn,
name = db_table,
schema = db_schema,
if_exists = 'replace',
index = False,
chunksize=1000,
method='multi' )

In [ ]:
create_mukey_county_runs(greene_soils_list, dbconn, 'sfc', 'Greene', 'IA073')
create_mukey_county_runs(greene_soils_list, dbconn, 'cc', 'Greene', 'IA073')
create_mukey_county_runs(greene_soils_list, dbconn, 'cfs', 'Greene', 'IA073')

In [ ]:
all_output = parse_all_output("C:\\Users\\mnowatz\\Documents\\Dev\\aepe\\analyses\\apsim_files\\Greene")

In [ ]:
summary_output = parse_summary_output("C:\\Users\\mnowatz\\Documents\\Dev\\aepe\\analyses\\apsim_files\\Greene")

In [ ]:
db_schema = 'raccoon'
db_table = 'apsim_output_summary' 
summary_output.to_sql(
con = dbconn,
name = db_table,
schema = db_schema,
if_exists = 'replace',
index = False,
chunksize=1000,
method='multi' )

In [ ]:
db_schema = 'raccoon'
db_table = 'apsim_output_all'
all_output.to_sql(
con = dbconn,
name = db_table,
schema = db_schema,
if_exists = 'replace',
index = False,
chunksize=1000,
method='multi' )

In [ ]:
#group by clukeys
#get clukey rotation
#look up yield for each mukey within clukey
#calculate profit
#append profit to clukey df

In [ ]:
output_table = 'raccoon.apsim_output_summary'
county = 'Greene'
#county_df = pd.read_sql(f"SELECT * FROM raccoon.raccoon_clu_ssurgo_2018 WHERE county = '{county}';", dbconn)
county_df = gpd.read_postgis(f"SELECT * FROM raccoon.raccoon_clu_ssurgo_2018 WHERE county = '{county}' LIMIT 10;", dbconn, geom_col='wkb_geometry')

In [ ]:
clukeys = list(county_df['clukey'].drop_duplicates())
#clukey_df = county_df.loc[county_df['clukey'] == 2516788]
#clukey_df.head()

In [ ]:
costs = pd.read_sql(f"SELECT * FROM public.crop_prod_costs WHERE year = '2018'", dbconn)

In [ ]:
def calc_subfield_profit(clukey_list, county_df, year, cfs_mgmt, sfc_mgmt, cc_mgmt, rent=False, labor=0, labor_payrate=0):
    df_rows = []
    total_clukeys = len(clukey_list)
    mukey_counter = 0
    clukey_counter = 0
    #get commodity prices
    comm_price_df = pd.read_sql(f"SELECT * from public.hist_comm_prices WHERE year = '{year}';", dbconn)
    corn_price = float(comm_price_df.loc[comm_price_df['commodity'] == 'corn', 'mrkt_avg'].iloc[0])
    soy_price = float(comm_price_df.loc[comm_price_df['commodity'] == 'soybean', 'mrkt_avg'].iloc[0])
    #get input costs
    costs = pd.read_sql(f"SELECT * FROM public.crop_prod_costs WHERE year = '{year}'", dbconn)
    corn_seed_price = float(costs.loc[costs['input'] == 'corn_seed', 'fixed-cost'].iloc[0])
    soy_seed_price = float(costs.loc[costs['input'] == 'soy_seed', 'fixed-cost'].iloc[0])
    n_price = float(costs.loc[costs['input'] == 'nitrogen', 'fixed-cost'].iloc[0])
    phos_price = float(costs.loc[costs['input'] == 'phosphorous', 'fixed-cost'].iloc[0])
    potash_price = float(costs.loc[costs['input'] == 'phosphorous', 'fixed-cost'].iloc[0])
    fix_chisel = float(costs.loc[costs['input'] == 'chisel_plow', 'fixed-cost'].iloc[0])
    var_chisel = float(costs.loc[costs['input'] == 'chisel_plow', 'variable-cost'].iloc[0])
    fix_disk = float(costs.loc[costs['input'] == 'disk_field_cultivator', 'fixed-cost'].iloc[0])
    var_disk = float(costs.loc[costs['input'] == 'disk_field_cultivator', 'variable-cost'].iloc[0])
    fix_applicator = float(costs.loc[costs['input'] == 'nh3_applicator', 'fixed-cost'].iloc[0])
    var_applicator = float(costs.loc[costs['input'] == 'nh3_applicator', 'variable-cost'].iloc[0])
    fix_cultivator = float(costs.loc[costs['input'] == 'field_cultivator', 'fixed-cost'].iloc[0])
    var_cultivator = float(costs.loc[costs['input'] == 'field_cultivator', 'variable-cost'].iloc[0])
    fix_planter = float(costs.loc[costs['input'] == 'planter', 'fixed-cost'].iloc[0])
    var_planter = float(costs.loc[costs['input'] == 'planter', 'variable-cost'].iloc[0])
    fix_sprayer = float(costs.loc[costs['input'] == 'sprayer', 'fixed-cost'].iloc[0])
    var_sprayer = float(costs.loc[costs['input'] == 'sprayer', 'variable-cost'].iloc[0])
    fix_corn_combine = float(costs.loc[costs['input'] == 'combine_corn', 'fixed-cost'].iloc[0])
    var_corn_combine = float(costs.loc[costs['input'] == 'combine_corn', 'variable-cost'].iloc[0])
    fix_soy_combine = float(costs.loc[costs['input'] == 'combine_soybeans', 'fixed-cost'].iloc[0])
    var_soy_combine = float(costs.loc[costs['input'] == 'combine_soybeans', 'variable-cost'].iloc[0])
    fix_wagon = float(costs.loc[costs['input'] == 'grain_cart', 'fixed-cost'].iloc[0])
    var_wagon = float(costs.loc[costs['input'] == 'grain_cart', 'variable-cost'].iloc[0])
    fix_hauling = float(costs.loc[costs['input'] == 'haul_grain', 'fixed-cost'].iloc[0])
    var_hauling = float(costs.loc[costs['input'] == 'haul_grain', 'variable-cost'].iloc[0])
    fix_drying = float(costs.loc[costs['input'] == 'dry_grain', 'fixed-cost'].iloc[0])
    var_drying = float(costs.loc[costs['input'] == 'dry_grain', 'variable-cost'].iloc[0])
    fix_handle = float(costs.loc[costs['input'] == 'auger_store_grain', 'fixed-cost'].iloc[0])
    var_handle = float(costs.loc[costs['input'] == 'auger_store_grain', 'variable-cost'].iloc[0])
    soy_herb_cost = float(costs.loc[(costs['crop'] == 'soy') & (costs['input'] == 'herbicide'), 'fixed-cost'].iloc[0])
    corn_herb_cost = float(costs.loc[(costs['crop'] == 'corn') & (costs['input'] == 'herbicide'), 'fixed-cost'].iloc[0])
    insect_cost = float(costs.loc[costs['input'] == 'insecticide', 'fixed-cost'].iloc[0])
    corn_crop_insurance = float(costs.loc[(costs['crop'] == 'corn') & (costs['input'] == 'crop_insurance'), 'variable-cost'].iloc[0])
    soy_crop_insurance = float(costs.loc[(costs['crop'] == 'soy') & (costs['input'] == 'crop_insurance'), 'variable-cost'].iloc[0])
    rent_cost = float(costs.loc[costs['input'] == 'rent', 'fixed-cost'].iloc[0])
    misc_cost = float(costs.loc[costs['input'] == 'miscellaneous', 'variable-cost'].iloc[0])
    for i in clukey_list:
        try:
            print(f'Getting data for clukey {i}')
            clukey_counter += 1
            rot_df = pd.read_sql(f"SELECT * FROM raccoon.raccoon_rots WHERE clukey = {i};", dbconn)
            clukey_df = county_df.loc[county_df['clukey'] == i].reset_index(drop=True)
            rotation = get_rotation(rot_df, 'crop')
            for i in clukey_df.index:
                try:
                    state = clukey_df.loc[i, 'state']
                    fips = clukey_df.loc[i, 'fips']
                    huc8 = clukey_df.loc[i, 'huc8']
                    county = clukey_df.loc[i, 'county']
                    geom = clukey_df.loc[i, 'wkb_geometry']
                    mukey = clukey_df.loc[i, 'mukey']
                    musym = clukey_df.loc[i, 'musym']
                    try:
                        mukey_summary = pd.read_sql(f"SELECT * FROM raccoon.apsim_output_summary WHERE county = '{county}' AND mukey = {mukey} AND rotation = '{rotation}';", dbconn)
                        if mukey_summary.empty:
                            continue
                    except Exception:
                        traceback.print_exc()
                        continue
                    acreage = float(clukey_df.loc[i, 'acres'])
                    no3_loss = float((mukey_summary.iloc[0]['leach_no3']/2.471)*acreage)
                    clukey = clukey_df.loc[i, 'clukey']
                    if rotation == 'cc':
                        current_crop = 'corn'
                        buac = float(mukey_summary.iloc[0]['corn_buac'])
                        n_lbs = float(mukey_summary.iloc[0]['fertiliser'])*0.892
                        #calculate how much phos and potash to add based on yield
                        phos_lbs = buac*0.37
                        potash_lbs = buac*0.37
                        #plant density is in m2 in apsim -- 10000 sq meters in hectare, 2.47 acres in hectare -- seeding rate = plant density * 10000 / 2.47
                        sowing_density = (int(cc_mgmt['sowing_density'])*10000)/2.47
                        cc_budget = Budget(rotation, current_crop, buac, acreage, year)
                        fert_cost = cc_budget.fert_cost(n_lbs, n_price, phos_lbs, phos_price, potash_lbs, potash_price)
                        seed_cost = cc_budget.seed_cost(sowing_density, corn_seed_price)
                        preharvest_machinery_cost = cc_budget.preharvest_machinery_cost(
                            fix_chisel=fix_chisel,
                            var_chisel=var_chisel,
                            fix_disk=fix_disk,
                            var_disk=var_disk,
                            fix_applicator=fix_applicator,
                            var_applicator=var_applicator,
                            fix_planter=fix_planter,
                            var_planter=var_planter,
                            fix_sprayer=fix_sprayer,
                            var_sprayer=var_sprayer)
                        harvest_machinery_cost = cc_budget.harvest_machinery_cost(fix_corn_combine, var_corn_combine, fix_wagon, var_wagon)
                        processing_cost = cc_budget.processing_cost(fix_hauling, var_hauling, fix_drying, var_drying, fix_handle, var_handle)
                        chemicals_cost = cc_budget.chemicals_cost(corn_herb_cost, insect_cost)
                        extra_costs = cc_budget.extra_costs(misc_cost, corn_crop_insurance)
                        labor_cost = cc_budget.labor_cost()
                        revenue = cc_budget.calc_revenue(corn_price)
                        expenses = cc_budget.sum_expenses()
                        profit = cc_budget.calc_profit()
                        df_rows.append((
                            state,
                            huc8,
                            fips,
                            county,
                            clukey,
                            mukey,
                            musym,
                            acreage,
                            rotation,
                            current_crop,
                            buac,
                            no3_loss,
                            n_lbs,
                            phos_lbs,
                            potash_lbs,
                            sowing_density,
                            fert_cost,
                            seed_cost,
                            preharvest_machinery_cost,
                            harvest_machinery_cost,
                            processing_cost,
                            chemicals_cost,
                            extra_costs,
                            labor_cost,
                            revenue,
                            expenses,
                            profit,
                            geom)
                        )
                    elif rotation == 'cfs':
                        current_crop = 'corn'
                        buac = float(mukey_summary.iloc[0]['corn_buac'])
                        n_lbs = float(mukey_summary.iloc[0]['fertiliser'])*0.892
                        #calculate how much phos and potash to add based on yield
                        phos_lbs = buac*0.375
                        potash_lbs = buac*0.3
                        sowing_density = (int(cfs_mgmt['sowing_density'])*10000)/2.47
                        cfs_budget = Budget(rotation, current_crop, buac, acreage, year)
                        fert_cost = cfs_budget.fert_cost(n_lbs, n_price, phos_lbs, phos_price, potash_lbs, potash_price)
                        seed_cost = cfs_budget.seed_cost(sowing_density, corn_seed_price)
                        preharvest_machinery_cost = cfs_budget.preharvest_machinery_cost(
                            fix_chisel=fix_chisel,
                            var_chisel=var_chisel,
                            fix_disk=fix_disk,
                            var_disk=var_disk,
                            fix_applicator=fix_applicator,
                            var_applicator=var_applicator,
                            fix_planter=fix_planter,
                            var_planter=var_planter,
                            fix_sprayer=fix_sprayer,
                            var_sprayer=var_sprayer)
                        harvest_machinery_cost = cfs_budget.harvest_machinery_cost(fix_corn_combine, var_corn_combine, fix_wagon, var_wagon)
                        processing_cost = cfs_budget.processing_cost(fix_hauling, var_hauling, fix_drying, var_drying, fix_handle, var_handle)
                        chemicals_cost = cfs_budget.chemicals_cost(corn_herb_cost)
                        extra_costs = cfs_budget.extra_costs(misc_cost, corn_crop_insurance)
                        labor_cost = cfs_budget.labor_cost()
                        revenue = cfs_budget.calc_revenue(corn_price)
                        expenses = cfs_budget.sum_expenses()
                        profit = cfs_budget.calc_profit()
                        df_rows.append((
                            state,
                            huc8,
                            fips,
                            county,
                            clukey,
                            mukey,
                            musym,
                            acreage,
                            rotation,
                            current_crop,
                            buac,
                            no3_loss,
                            n_lbs,
                            phos_lbs,
                            potash_lbs,
                            sowing_density,
                            fert_cost,
                            seed_cost,
                            preharvest_machinery_cost,
                            harvest_machinery_cost,
                            processing_cost,
                            chemicals_cost,
                            extra_costs,
                            labor_cost,
                            revenue,
                            expenses,
                            profit,
                            geom)
                        )
                    elif rotation == 'sfc':
                        current_crop = 'soybean'
                        buac = float(mukey_summary.iloc[0]['soy_buac'])
                        #calculate how much phos and potash to add based on yield
                        phos_lbs = buac*0.8
                        potash_lbs = buac*1.51
                        n_lbs = 0
                        sowing_density = (int(sfc_mgmt['sowing_density'])*10000)/2.47
                        spray_fix = fix_sprayer*2
                        spray_var = var_sprayer*2
                        sfc_budget = Budget(rotation, current_crop, buac, acreage, year)
                        fert_cost = sfc_budget.fert_cost(n_lbs, n_price, phos_lbs, phos_price, potash_lbs, potash_price)
                        seed_cost = sfc_budget.seed_cost(sowing_density, soy_seed_price)
                        preharvest_machinery_cost = sfc_budget.preharvest_machinery_cost(
                            fix_chisel=fix_chisel,
                            var_chisel=var_chisel,
                            fix_disk=fix_disk,
                            var_disk=var_disk,
                            fix_applicator=fix_applicator,
                            var_applicator=var_applicator,
                            fix_planter=fix_planter,
                            var_planter=var_planter,
                            fix_sprayer=spray_fix,
                            var_sprayer=spray_var)
                        harvest_machinery_cost = sfc_budget.harvest_machinery_cost(fix_soy_combine, var_soy_combine, fix_wagon, var_wagon)
                        processing_cost = sfc_budget.processing_cost(fix_hauling, var_hauling, fix_drying, var_drying, fix_handle, var_handle)
                        chemicals_cost = sfc_budget.chemicals_cost(soy_herb_cost)
                        extra_costs = sfc_budget.extra_costs(misc_cost, soy_crop_insurance)
                        labor_cost = sfc_budget.labor_cost()
                        revenue = sfc_budget.calc_revenue(soy_price)
                        expenses = sfc_budget.sum_expenses()
                        profit = sfc_budget.calc_profit()
                        df_rows.append((
                            state,
                            huc8,
                            fips,
                            county,
                            clukey,
                            mukey,
                            musym,
                            acreage,
                            rotation,
                            current_crop,
                            buac,
                            no3_loss,
                            n_lbs,
                            phos_lbs,
                            potash_lbs,
                            sowing_density,
                            fert_cost,
                            seed_cost,
                            preharvest_machinery_cost,
                            harvest_machinery_cost,
                            processing_cost,
                            chemicals_cost,
                            extra_costs,
                            labor_cost,
                            revenue,
                            expenses,
                            profit,
                            geom)
                        )
                    else:
                        print('Not a corn or soybean rotation.')
                        continue
                    mukey_counter += 1
                    if mukey_counter % 50 == 0:
                        print(f"Finished with {mukey_counter} mukeys")
                    if clukey_counter == total_clukeys:
                        print("All done")
                except Exception:
                    traceback.print_exc()
                    print(f'failed on mukey {mukey}')
                    mukey_counter +=1
                    if mukey_counter % 50 == 0:
                        print(f"Finished with {mukey_counter} mukeys")
                    if clukey_counter == total_clukeys:
                        print("All done")
                    continue
        except Exception:
            traceback.print_exc()
            continue
    return gpd.GeoDataFrame(df_rows,
    #return pd.DataFrame(df_rows,
                        columns=(
                            "state",
                            "huc8",
                            "fips",
                            "county",
                            "clukey",
                            "mukey",
                            "musym",
                            "acreage",
                            "rotation",
                            "crop",
                            "buac",
                            "no3_loss",
                            "n_lbs",
                            "phos_lbs",
                            "potash_lbs",
                            "sowing_density",
                            "fert_cost",
                            "seed_cost",
                            "preharv_machinery",
                            "harv_machinery",
                            "processing",
                            "chemicals",
                            "extra_costs",
                            "labor_cost",
                            "revenue",
                            "expenses",
                            "profit",
                            "wkb_geometry"
                        ), geometry="wkb_geometry")

In [ ]:
greene_profit = calc_subfield_profit(clukeys, county_df, 2018, cfs_mgmt=cfs_mgmt, sfc_mgmt=sfc_mgmt, cc_mgmt=cc_mgmt, rent=False):

In [ ]:
greene_profit.to_file("greene_profit.json", driver="GeoJSON")
#.from_postgis("SELECT * FROM raccoon.subfield_profit;", dbconn, geom_col='wkb_geometry')

In [ ]:
# db_schema = 'raccoon'
# db_table = 'subfield_profit'
# greene_profit.to_sql(
# con = dbconn,
# name = db_table,
# schema = db_schema,
# if_exists = 'replace',
# index = False,
# chunksize=1000,
# method='multi' )

In [ ]:
profit_df = gpd.read_file("greene_profit.json")
#profit_df.head()

In [ ]:
profit_df = profit_df[['clukey','mukey','rotation','acreage','buac','profit','no3_loss','geometry']]
profit_df['mukey'] = profit_df.mukey.astype(int)

In [ ]:
cfs_profit_df = profit_df.loc[profit_df['rotation'] == 'cfs']
cc_profit_df = profit_df.loc[profit_df['rotation'] == 'cc']
sfc_profit_df = profit_df.loc[profit_df['rotation'] == 'sfc']

In [ ]:
cfs_lower = cfs_profit_df["profit"].quantile(0.01)
cfs_upper = cfs_profit_df["profit"].quantile(0.99)

In [ ]:
cfs_filtered = cfs_profit_df[(cfs_profit_df["profit"] < cfs_upper) & (cfs_profit_df["profit"] > cfs_lower)]

In [ ]:
#sns.distplot(cfs_filtered['profit'], hist=True, kde=False, bins=30, color = 'blue', hist_kws={'edgecolor':'black'})

In [ ]:
x_coord = profit_df.centroid.x.mean()
y_coord = profit_df.centroid.y.mean()
print(x_coord, y_coord)

In [ ]:
#greene county lat long
greene_coords = (y_coord, x_coord)
greene_map = folium.Map(greene_coords, zoom_start=12)

In [ ]:
myscale = (cfs_filtered['profit'].quantile((0,0.1,0.25,0.5,0.75,0.9,0.98,1))).tolist()
folium.features.Choropleth(
    geo_data=cfs_filtered,
    name="Corn following soy profitability (acres owned)",
    data=cfs_filtered,
    columns=('mukey', 'profit'),
    key_on='feature.properties.mukey',
    threshold_scale=myscale,
    legend_name='Field profitability given soil'

).add_to(greene_map)
# no3_bins = list(cfs_filtered['no3_loss'].quantile((0,0.1,0.25,0.5,0.75,0.9,0.98,1)))
# folium.features.Choropleth(
#     geo_data=cfs_filtered,
#     name="NO3 leaching",
#     data=cfs_filtered,
#     columns=('mukey', 'no3_loss'),
#     legend_name='Nitrate-nitrogen loss',
#     key_on='feature.properties.mukey'

# ).add_to(greene_map)
folium.LayerControl().add_to(greene_map)